Данная статья интересна тем, что в ней впервые (насколько я смог найти) применяется операция транспонированной свертки. Статья написана на основе статьи [Deconvolutional Networks]($Deconvolutional Networks$), и в ней рассматривается сверточная сеть, в которой информация распространяется в двух направлениях. Такая сеть применяется для задачи обучения на изображениях без разметки (self-supervised learning).

Схема сети приведена на иллюстрации (слева):

<img src="assets/deconv2.jpg" width="900" align="center">

### Устройство сети

**Reconstructed input** $\hat{y}$ (снизу) - это изображение. Оно получается операцией свертки ($F_1$) из карты признаков первого слоя (**Layer 1 features $z_1$**). Обратное отображение осуществляется транспонированной сверткой с теми же весами ($F_1^T$).

*Примечание от автора обзора. В статье не до конца объясняется правомерность таких операций. Транспонированная свертка в общем случае не является обратным преобразованием. Она будет обратным преобразованием только тогда, когда матрица сверки ортогональна, но я не смог найти в статье упоминания о таком свойстве. Говорится лишь то, что матрица свертки "нормализуется до единичной длины".*

Следующей операцией является **max pooling $P_{s_1}$**, причем отуществляется он не только по пространственным осям, но и по оси канала, и имеет шаг и размер окна 2x2x2. Из каждого окна выбирается максимальный *по модулю* элемент. При этом также сохраняются пространственные позиции выбранных элементов (для каждой позиции окна - 3 координаты выбранного элемента внутри окна). Сохраненные позиции хранятся в массиве **L1 switches $s_1$**. Эти позиции используются в операции **unpooling $U_{s_1}$**. Пример работы операций max pooling и unpooling с использованием switches показан на изображении справа.

*Примечание. В последующей статье [Visualizing and Understanding Convolutional Networks, 2014]($Visualizing and Understanding Convolutional Networks$) авторы будут применять такой же подход для попытки визуализации и интерпретации глубоких сверточных сетей, получивших распространение после появления [AlexNet]($ImageNet Classification with Deep Convolutional Neural Networks$).*

Операции unpooling и свертка, а также действующие в обратную сторону операции транспонированной свертки и max pooling составляют слой сети. Таких слов в сети может быть несколько (авторы используют до четырех слоев). Также определяются операторы проекции, действующие между входным изображением и i-м слоем, например $R_2$ и $R_2^T$ действуют между картой признаков второго слоя и исходным изображением. Оператор $R_i^T$ вместо max pooling используют заранее полученные switches (то есть оператор $R_i^T$ принимает на вход input $\hat{y}$ и switches, и возвращает $z_i$. Поскольку между слоями нет нелинейностей, а switches даны заранее, то операторы $R_2$ и $R_2^T$ являются линейными.

### Обучение

Вся сеть обучается послойно на датасете из неразмеченных изображений. Процесс обучения состоит из прямых и обратных проходов по слою и в обзоре рассматриваться не будет (см. разделы 2.1, 2.2 и Algorithm 1 в статье).

В ходе обучения оптимизируются одновременно веса сверточных слоев и карты признаков для всех изображений из обучающего датасета (как в статье [Deconvolutional Networks]($Deconvolutional Networks$)). При обучении на карты признаков $z_i$ накладывается L1-регуляризация, а также применяется операция shrinkage, которая зануляет те элементы $z_i$, которые близки к нулю (их модули ниже заданного порога).

### Инференс и визуализация

После обучения модель может использоваться для инференса, при котором для произвольного изображения расчитываются карты признаков. Алгоритм инференса описан в разделе 2.1 статьи. Полученную на одном из слоев сети карту признаков можно использовать для классификации. Кроме того, в карте признаков сети $z_i$ можно найти несколько наибольших по модулю значений и спроецировать их на исходное изображение. Для этого все остальные значения, кроме одного, зануляются в карте признаков $z_i$. Для проекции используется оператор $R_i$, который использует набор switches, полученный при инференсе. Тем самым можно сделать попытку визуализации и интерпретации элементов карты признаков. Аналогичный подход авторы будут использовать в статье [Visualizing and Understanding Convolutional Networks]($Visualizing and Understanding Convolutional Networks$).

Например, три наибольшие активации 4-го слоя модели проецируются на исходное изображение таким образом:

<img src="assets/deconv3.jpg" width="600" align="center">